In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui

# Mediapipe setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Screen resolution
screen_width, screen_height = pyautogui.size()

# Volume adjustment tracking
prev_x = 0  # Tracks the wrist's horizontal position for volume adjustment

# Gesture detection helper functions
def is_finger_up(hand_landmarks, finger_idx):
    """Check if a specific finger is up (extended)."""
    return hand_landmarks[finger_idx][1] < hand_landmarks[finger_idx - 2][1]

def are_all_fingers_open(hand_landmarks):
    """Check if all fingers are open (extended)."""
    return all(hand_landmarks[i][1] < hand_landmarks[i - 2][1] for i in [8, 12, 16, 20])  # Index, middle, ring, pinky

def are_fingers_closed_except(hand_landmarks, finger_indices):
    """Check if all fingers except specified indices are closed."""
    closed = True
    for i in [8, 12, 16, 20]:  # Index, middle, ring, and pinky
        if i not in finger_indices and hand_landmarks[i][1] < hand_landmarks[i - 2][1]:
            closed = False
    return closed

def are_two_fingers_together(hand_landmarks, idx1, idx2):
    """Check if two fingers are touching (close together)."""
    distance = np.linalg.norm(np.array(hand_landmarks[idx1]) - np.array(hand_landmarks[idx2]))
    return distance < 30  # Adjust threshold as needed

# Webcam feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmarks.append((cx, cy))

            # Gesture 1: Cursor Movement (Index Finger Up, All Other Fingers Closed)
            if is_finger_up(landmarks, 8) and are_fingers_closed_except(landmarks, [8]):
                cursor_x = np.interp(landmarks[8][0], [0, w], [0, screen_width])
                cursor_y = np.interp(landmarks[8][1], [0, h], [0, screen_height])
                pyautogui.moveTo(cursor_x, cursor_y)

            # Gesture 2: Volume Adjustment (All Fingers Closed, Hand Left/Right)
            elif are_fingers_closed_except(landmarks, []):  # No fingers up
                current_x = landmarks[0][0]  # Use wrist's horizontal position
                if prev_x:
                    if current_x > prev_x + 20:  # Hand moved right
                        pyautogui.press('volumeup')
                        print("Volume Up")
                    elif current_x < prev_x - 20:  # Hand moved left
                        pyautogui.press('volumedown')
                        print("Volume Down")
                prev_x = current_x

            # Gesture 3: Screenshot (Index & Middle Fingers Together, Others Closed)
            elif are_two_fingers_together(landmarks, 8, 12) and are_fingers_closed_except(landmarks, [8, 12]):
                pyautogui.screenshot("screenshot.png")
                print("Screenshot taken!")

            # Gesture 4: Click (All Fingers Open)
            elif are_all_fingers_open(landmarks):
                pyautogui.click()
                print("Mouse clicked!")

            # Draw landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the webcam feed
    cv2.imshow("Virtual Mouse", frame)

    # Exit with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Volume Down
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Mouse clicked!
Volume Down
Volume Down
Volume Down
Volume Down
Volume Up
Volume Up
Volume Down
Volume Down
Volume Up
Volume Up
Volume Up
Volume Up
Volume Up
Screenshot taken!
Screenshot taken!
Screenshot taken!
Screenshot taken!
Volume Down
Volume Down
Mouse clicked!
Mouse clicked!
Mouse clicked!
